In [20]:
# !pip install --user --upgrade catboost
# !pip install --user --upgrade ipywidgets
# !pip install shap
# !pip install sklearn
# !pip install --upgrade numpy
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [21]:
!pip install --pre deepchem
!pip install --user --upgrade catboost
!pip install --user --upgrade ipywidgets
!pip install shap
!pip install sklearn
!pip install --upgrade numpy
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [22]:
import deepchem as dc
import pandas as pd
from deepchem.molnet.load_function.molnet_loader import TransformerGenerator, _MolnetLoader

from deepchem.data import Dataset
from typing import List, Optional, Tuple, Union
import pandas as pd
import numpy as np
import random
import os
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import catboost
from catboost import *
from catboost import datasets
from catboost import CatBoostClassifier

In [23]:

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import catboost
from catboost import *
from catboost import datasets
feature_field="smiles"
TASKS = ['TOXRIC_Toxicity_Value', 'VirginiaSabando_Toxicity_Value']

In [24]:
task = "/Solution/Ames/train_ames.csv"
task_dir = "/Solution/Ames/"
task_save = "/Solution/Ames/"
df = pd.read_csv(task)
class MyLoader(_MolnetLoader):

    def create_dataset(self) -> Dataset:
        loader = dc.data.CSVLoader(tasks=self.tasks,
                                   feature_field=feature_field,
                                   featurizer=self.featurizer)
        return loader.create_dataset(task, shard_size=8192)


def load_tox21(
    featurizer: Union[dc.feat.Featurizer, str] = 'ECFP',
    splitter: Union[dc.splits.Splitter, str, None] = 'scaffold',
    transformers: List[Union[TransformerGenerator, str]] = ['balancing'],
    reload: bool = True,
    data_dir: Optional[str] = task_dir,
    save_dir: Optional[str] = task_save,
    **kwargs
) -> Tuple[List[str], Tuple[Dataset, ...], List[dc.trans.Transformer]]:
    
    loader = MyLoader(featurizer, splitter, transformers, TASKS,
                          data_dir, save_dir, **kwargs)
    return loader.load_dataset(task, reload)
tasks, datasets, transformers = load_tox21()
train_dataset, valid_dataset, test_dataset = datasets
df_tr = train_dataset.to_dataframe()

df_t = test_dataset.to_dataframe()
df_v = valid_dataset.to_dataframe()

In [26]:

x = df_tr.drop(['y1','w1','w2','ids'],axis=1)
y = df_tr['y1']
xt = df_t.drop(['y1','w1','w2','ids'],axis=1)
 
x_smiles = df_t['ids'].to_numpy()

yt = df_t['y1']
xv = df_v.drop(['y1','w1','w2','ids'],axis=1)

yv = df_v['y1']

In [27]:
def set_seed(s):
    seed_value= s
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    import random
    random.seed(seed_value)
    import numpy as np
    np.random.seed(seed_value)



In [28]:
set_seed(17)

In [29]:
import matplotlib.pyplot as plt
def show_history(hist):
    acc = hist.history['AUC']
    val_acc = hist.history['val_AUC']
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, '-', label='AUC')
    plt.plot(epochs, val_acc, ':', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.plot()

In [30]:
cat_features = list(range(x.shape[1]))

In [31]:
type(cat_features[0])

int

In [32]:
pool = Pool(data=x, label=y)


In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=100000,
    learning_rate=0.001,
    loss_function='CrossEntropy',
    eval_metric='AUC',
    early_stopping_rounds=2000,
    
    # weight=w
        # stratify=True


    # task_type="GPU"
)

model.fit(
    x, y,
    # cat_features=cat_features,
    eval_set=(xv, yv),
    verbose=False,
    plot=True,

)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
roc_auc_score(yt,model.predict_proba(xt)[:,1])

In [ ]:
pred = model.predict(xt)
df_savey = pd.DataFrame(data={'smiles':list(x_smiles)})
df_savey['y']=pred
df_savey.to_csv('output_classification_ames_cat.csv',index=False)  